# 01 MSCI Theme Select


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [2]:
from src.signals.theme_selection_msci import compute_theme_rank, compute_composite_return

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

selected_themes, theme_weights, debug_df = compute_theme_rank(
    theme_scores=theme_scores,
    theme_index_returns=returns,
    rebalance_dates=rebalance_dates,
    top_k=cfg["msci_theme_select"]["top_k"],
    stale_months=cfg["msci_theme_select"]["stale_months"],
    window_days=cfg["msci_theme_select"]["fallback"]["window_days"],
)

composite_ret = compute_composite_return(returns, theme_weights, lag=cfg["calendar"]["trading_lag_days"])
weights_applied = theme_weights.shift(1)
assert weights_applied.iloc[0].isna().all()
active_weights = theme_weights.loc[theme_weights.sum(axis=1) > 0]
assert_weights_sum_to_one(active_weights)

assert selected_themes.shape[0] > 0
assert theme_weights.shape == returns.shape
assert not debug_df.empty

save_parquet(selected_themes, PROJECT_ROOT / "outputs/signals/01_theme_rank.parquet")
save_parquet(theme_weights, PROJECT_ROOT / "outputs/weights/01_theme_weights_raw.parquet")
save_csv(debug_df, PROJECT_ROOT / "outputs/diagnostics/01_theme_selection_debug.csv")

fig, ax = plt.subplots(figsize=(10, 4))
composite_ret.fillna(0.0).cumsum().plot(ax=ax)
ax.set_title("01 Composite Return")
ax.set_ylabel("Cumulative Return")
save_figure(fig, PROJECT_ROOT / "outputs/figures/01_theme_rank.png")
plt.close(fig)

print("selected_themes", selected_themes.shape)
print("theme_weights", theme_weights.shape)
print("debug_df", debug_df.shape)


selected_themes (192, 4)
theme_weights (4174, 6)
debug_df (192, 7)


In [3]:
required = [
    "outputs/signals/01_theme_rank.parquet",
    "outputs/weights/01_theme_weights_raw.parquet",
    "outputs/diagnostics/01_theme_selection_debug.csv",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=01_msci_theme_select.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)


Notebook=01_msci_theme_select.ipynb, OK, 生成ファイル確認=outputs/signals/01_theme_rank.parquet:True, outputs/weights/01_theme_weights_raw.parquet:True, outputs/diagnostics/01_theme_selection_debug.csv:True

